In [7]:
import pandas as pd

In [8]:
sp = pd.read_csv('sp.csv')
madrid = pd.read_csv('madrid.csv')
dhaka = pd.read_csv('dhaka.csv', index_col=[0])

In [12]:
madrid.drop('Link', axis=1, inplace=True)
madrid.drop('Summary', axis=1, inplace=True)
madrid

ValueError: Cannot specify both 'labels' and 'index'/'columns'